# Reprotrack interactive session - Travel mode split

Travel mode choices are formed due to the everyday needs and constraints
 of individuals and are influenced by travel-related factors such as 
cost, time, accessibility, and comfort. It is, therefore, not uncommon 
for mobility systems to be evaluated and compared based on the current 
modal splits of the overall population. Trackintel provides a series of 
travel mode related functions to assist in mode analysis of individual movement. 



### Import dependencies and libraries

In [ ]:
import os
import json
import pickle as pickle

import geopandas as gpd
import pandas as pd
import numpy as np

# for fetching osm data
import osmnx as ox

# for visualization
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# trackintel -> the functions will be imported with full names
import trackintel as ti


In [ ]:
# options for better printing and visualizing

colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]

matplotlib.rcParams["figure.dpi"] = 300
matplotlib.rcParams["xtick.labelsize"] = 13
matplotlib.rcParams["ytick.labelsize"] = 13
np.set_printoptions(precision=4)
np.set_printoptions(suppress=True)

%matplotlib inline

In [ ]:
# get the data dir from the config file -> default stored in Data/Geolife/Data folder

DATA_DIR = os.path.join("..", "paths.json")
with open(DATA_DIR) as json_file:
    CONFIG = json.load(json_file)

save_dir = os.path.join("..", CONFIG["data_dir"])

# Load Geolife data
You can use your own dataset (see instructions below), or work with the Geolife dataset. In the latter case, Trackintel provides a function [read_geolife](https://trackintel.readthedocs.io/en/latest/modules/io.html#trackintel.io.read_geolife) to directly load [Geolife](https://www.microsoft.com/en-us/research/publication/geolife-gps-trajectory-dataset-user-guide/) dataset into the trackintel positionfixes. For the tutorial and interactive section, we provided you a sample dataset with 20 selected users from the geolife dataset. This can be directly loaded from the `save_dir`. 

In [ ]:

pfs, mode_labels = ti.io.read_geolife(save_dir, print_progress=True)

## Reading your own data
Trackintel provides an I/O module for accessing movement data and storing intermediate or final results in a file or database. Three methods for converting movement data with attached attribute information to Trackintel-compatible formats are provided:
1) reading from Pandas Dataframes and Geopandas Geodataframes (**recommended**)
2) reading and writing from CSV file formats
3) reading and storing from PostgreSQL databases with PostGIS extension. Check the [input/output](https://trackintel.readthedocs.io/en/latest/modules/io.html) module for more information. 

An important consideration before loading data is to find the appropriate movement level for your data. Depending on the semantics of the data, you should call different reading functions, e.g., [read_positionfixes_csv](https://trackintel.readthedocs.io/en/latest/modules/io.html#trackintel.io.read_positionfixes_csv) or [read_staypoints_csv](https://trackintel.readthedocs.io/en/latest/modules/io.html#trackintel.io.read_staypoints_csv). We provide a reference of the different data types in the figure below:

![Trackintel data support](prepare/trackintel_structure.png)

In [ ]:
# TODO: use ti.read_positionfixes_csv() or ti.read_staypoints_csv() for loading your data. Check the documentation for the required input parameters
# selected_pfs = ti.read_positionfixes_csv()

### Task 1: Generate triplegs

To compute a mode split, we first need to generate triplegs. For some datasets, you might have triplegs already and can load them directly with `ti.read_triplegs_csv`. Otherwise, we need to generate them with the `generate_staypoints` and `generate_triplegs` functions in Trackintel:


In [ ]:
# TODO: generate staypoints and triplegs


## Task 2: Add existing mode labels

If the transport mode is known in your dataset, we want to add this information to the triplegs GeoDataFrame with the attribute name `mode`. For Geolife, the labels can be attached using the Trackintel function [geolife_add_modes_to_triplegs](https://trackintel.readthedocs.io/en/latest/modules/io.html#trackintel.io.geolife_add_modes_to_triplegs). If you check out the documentation of the [read_geolife](https://trackintel.readthedocs.io/en/latest/modules/io.html#trackintel.io.read_geolife) function that we used above, you can see that this function returns a dictionary `mode_labels`. This dictionary can be used in the `geolife_add_to_triplegs` function.


In [ ]:
# TODO: assign the true mode labels to triplegs, with the help of trackintel functions 


## Task 3: Check for missing values

In Geolife, some triplegs have labelled modes, but many are still unknown. Print the ratio of missing values in the `mode` column (if there are no mode labels available for your dataset, this would be 100%).

In [ ]:
# TODO: check how many missing values are in the mode column


## Task 4: Impute missing values

We now want to use the Trackintel function [predict_transport_mode](geolife_add_modes_to_triplegs) to impute missing mode labels. This function is simply based on the average speed of a tripleg. Use the function to create predicted modes for all triplegs, and fill the missing values with these predicted modes.

In [ ]:
# TODO: impute transport mode with prediction based on speed
tpls_with_predicted_mode = ...
# add to the original dataframe as a column named "pred_mode"
tpls["pred_mode"] = tpls_with_predicted_mode["mode"]
# TODO: Fill missing values in "mode" column with the predicted modes

## Task 5: Align mode labels

You may have noted that the predicted modes have three categories: "slow", "motorized", and "fast". This is not comparable to the original Geolife categories (or the categories available in your own dataset). Use the following dictionary to rename the modes, such that all of them are one of "slow", "motorized", and "fast". Then count the number of triplegs per category.

In [ ]:
mode_to_category = {
    "fast_mobility": "fast",
    "car": "motorized",
    "bus": "motorized",
    "taxi": "motorized",
    "ecar": "motorized",
    "motorized_mobility": "motorized",
    "bike": "slow",
    "walk": "slow",
    "slow_mobility": "slow",
}

# TODO: change mode labels with the above mapping

## Task 6: Compute the modal split

Use the function [calculate_modal_split](https://trackintel.readthedocs.io/en/latest/modules/analysis.html#trackintel.analysis.calculate_modal_split) to compute the model split by count, distance or duration. Aggregate the data per month. Check out the other parameters of the function; for example, the modal split can be provided in absolute values or relative (`norm=True`).

Then, plot the modal split via the [plot_modal_split](https://trackintel.readthedocs.io/en/latest/modules/visualization.html#trackintel.plot_modal_split) function. 

In [ ]:
mode = "count" # modal split by count (number of triplegs per mode), can be switched for "distance" or "duration"

# TODO: calculate modal split
modal_split = ti.analysis.calculate_modal_split(tpls, ...)

# TODO: visualize the modal split
fig, ax = plt.subplots(figsize=(10, 6))
ax = ti.plot_modal_split(
    ...
    axis=ax,
    borderaxespad=2,
)
fig.autofmt_xdate()